In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
import warnings
warnings.filterwarnings("ignore")


df = pd.read_csv(r"/home/daksh/Downloads/pumpfun_09-08-2025_to_09-11-2025.csv/pumpfun_09-08-2025_to_09-11-2025.csv",engine='pyarrow').drop(columns=['Block ID', 'Transaction ID','Wallet Address', 'Platform'])
df

,Date,Token Address,Token Price,Token Amount,Transaction Type,SOL in Curve,Outstanding Shares
0,2025-09-10 21:53:07+00:00,8n2XWSL8mHX3mog9hSSPWeF4fX6bvYNKHEtdpiGxpump,2.842498e-08,1.759017e+07,Buy,0.500003,1.759026e+07
1,2025-09-10 21:53:07+00:00,8n2XWSL8mHX3mog9hSSPWeF4fX6bvYNKHEtdpiGxpump,3.435632e-08,1.676548e+08,Buy,6.260004,1.852450e+08
2,2025-09-10 21:53:07+00:00,8n2XWSL8mHX3mog9hSSPWeF4fX6bvYNKHEtdpiGxpump,4.687104e-08,1.141430e+08,Buy,11.610006,2.993881e+08
3,2025-09-10 21:53:07+00:00,8n2XWSL8mHX3mog9hSSPWeF4fX6bvYNKHEtdpiGxpump,6.008180e-08,8.105619e+07,Buy,16.480008,3.804443e+08
4,2025-09-10 21:53:07+00:00,8n2XWSL8mHX3mog9hSSPWeF4fX6bvYNKHEtdpiGxpump,7.327930e-08,5.827023e+07,Buy,20.750011,4.387145e+08
...,...,...,...,...,...,...,...
9714734,2025-09-12 18:56:22+00:00,W8A65eCkht8kQNQfrjv8CCKUtAD9DwkVPYn9tdqpump,6.282314e-08,3.991902e+06,Buy,15.095302,3.591785e+08
9714735,2025-09-12 20:17:17+00:00,W8A65eCkht8kQNQfrjv8CCKUtAD9DwkVPYn9tdqpump,6.166053e-08,4.414409e+05,Sell,14.538079,3.502477e+08
9714736,2025-09-12 20:46:16+00:00,W8A65eCkht8kQNQfrjv8CCKUtAD9DwkVPYn9tdqpump,6.160093e-08,4.170142e+04,Sell,14.528865,3.500982e+08
9714737,2025-09-12 21:17:04+00:00,W8A65eCkht8kQNQfrjv8CCKUtAD9DwkVPYn9tdqpump,6.159204e-08,3.742047e+04,Sell,14.525784,3.500482e+08


In [3]:
from statsmodels.tsa.stattools import adfuller
from statsmodels.stats.diagnostic import acorr_ljungbox
import seaborn as sns

def adf_test(series, title='Time Series'):
    """
    Perform Augmented Dickey-Fuller test for stationarity
    
    Parameters:
    series: pandas Series or array-like
        The time series to test
    title: str
        Title for the test results
        
    Returns:
    dict: Dictionary containing test results
    """
    print(f'Augmented Dickey-Fuller Test for {title}')
    print('=' * 50)
    
    result = adfuller(series.dropna(), autolag='AIC')
    
    print(f'ADF Statistic: {result[0]:.6f}')
    print(f'p-value: {result[1]:.6f}')
    print(f'Critical Values:')
    
    for key, value in result[4].items():
        print(f'\t{key}: {value:.3f}')
    
    # Interpretation
    if result[1] <= 0.05:
        print(f'\nResult: p-value ({result[1]:.6f}) <= 0.05')
        print('The series is STATIONARY (reject null hypothesis)')
    else:
        print(f'\nResult: p-value ({result[1]:.6f}) > 0.05')
        print('The series is NON-STATIONARY (fail to reject null hypothesis)')
    
    print('=' * 50)
    
    return {
        'adf_statistic': result[0],
        'p_value': result[1],
        'critical_values': result[4],
        'is_stationary': result[1] <= 0.05
    }

def plot_price_series(df, token_address, max_tokens=5):
    """
    Plot price series for multiple tokens
    """
    unique_tokens = df['Token Address'].unique()[:max_tokens]
    
    fig, axes = plt.subplots(len(unique_tokens), 1, figsize=(15, 4*len(unique_tokens)))
    if len(unique_tokens) == 1:
        axes = [axes]
    
    for i, token in enumerate(unique_tokens):
        token_data = df[df['Token Address'] == token].copy()
        token_data = token_data.sort_values('Date')
        
        axes[i].plot(token_data['Date'], token_data['Token Price'], linewidth=1)
        axes[i].set_title(f'Token Price Series: {token[:20]}...')
        axes[i].set_xlabel('Date')
        axes[i].set_ylabel('Token Price')
        axes[i].tick_params(axis='x', rotation=45)
        axes[i].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Display basic info about the dataset
print("Dataset Info:")
print(f"Shape: {df.shape}")
print(f"Date range: {df['Date'].min()} to {df['Date'].max()}")
print(f"Number of unique tokens: {df['Token Address'].nunique()}")
print(f"Transaction types: {df['Transaction Type'].value_counts().to_dict()}")
print("\nFirst few rows:")
df.head()


Dataset Info:
Shape: (9714739, 7)
Date range: 2025-01-10 17:37:45+00:00 to 2025-09-12 23:02:10+00:00
Number of unique tokens: 675
Transaction types: {'Buy': 5437011, 'Sell': 4277728}

First few rows:


,Date,Token Address,Token Price,Token Amount,Transaction Type,SOL in Curve,Outstanding Shares
0,2025-09-10 21:53:07+00:00,8n2XWSL8mHX3mog9hSSPWeF4fX6bvYNKHEtdpiGxpump,2.842498e-08,1.759017e+07,Buy,0.500003,1.759026e+07
1,2025-09-10 21:53:07+00:00,8n2XWSL8mHX3mog9hSSPWeF4fX6bvYNKHEtdpiGxpump,3.435632e-08,1.676548e+08,Buy,6.260004,1.852450e+08
2,2025-09-10 21:53:07+00:00,8n2XWSL8mHX3mog9hSSPWeF4fX6bvYNKHEtdpiGxpump,4.687104e-08,1.141430e+08,Buy,11.610006,2.993881e+08
3,2025-09-10 21:53:07+00:00,8n2XWSL8mHX3mog9hSSPWeF4fX6bvYNKHEtdpiGxpump,6.008180e-08,8.105619e+07,Buy,16.480008,3.804443e+08
4,2025-09-10 21:53:07+00:00,8n2XWSL8mHX3mog9hSSPWeF4fX6bvYNKHEtdpiGxpump,7.327930e-08,5.827023e+07,Buy,20.750011,4.387145e+08


In [ ]:
# Convert Date column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Let's test stationarity for a few different tokens
print("Testing Stationarity with ADF Test")
print("=" * 60)

# Get the most active tokens (by number of transactions)
token_counts = df['Token Address'].value_counts()
top_tokens = token_counts.head(3).index.tolist()

print(f"Testing top {len(top_tokens)} most active tokens:")
print(f"Token addresses: {top_tokens}")
print()

adf_results = {}

for i, token in enumerate(top_tokens):
    print(f"\n{'='*20} TOKEN {i+1} {'='*20}")
    token_data = df[df['Token Address'] == token].copy()
    token_data = token_data.sort_values('Date')
    
    print(f"Token: {token}")
    print(f"Number of transactions: {len(token_data)}")
    print(f"Price range: {token_data['Token Price'].min():.2e} to {token_data['Token Price'].max():.2e}")
    
    # Test original price series
    print(f"\n--- Testing Original Price Series ---")
    adf_results[f'{token}_original'] = adf_test(token_data['Token Price'], f'Token {i+1} Original Price')
    
    # Test log-transformed price series (often more stationary)
    print(f"\n--- Testing Log-Transformed Price Series ---")
    log_prices = np.log(token_data['Token Price'])
    adf_results[f'{token}_log'] = adf_test(log_prices, f'Token {i+1} Log Price')
    
    # Test first differences (price changes)
    print(f"\n--- Testing First Differences (Price Changes) ---")
    price_diff = token_data['Token Price'].diff().dropna()
    adf_results[f'{token}_diff'] = adf_test(price_diff, f'Token {i+1} Price Differences')
    
    print(f"\n{'='*50}")


Testing Stationarity with ADF Test
Testing top 3 most active tokens:
Token addresses: ['AcNia1KNYeaDAuswtpJJZpPqzpuFNyvjd7LbxMXSShji', 'CsKfV8ePhQWiyQxNJwXhKZHcmUyNWBkHFGrkZGdJpump', 'HFu8iR5Nro4Xeu4q8cxxR6ivBWSBd486w3fXdAuJpump']


==================== TOKEN 1 ====================
Token: AcNia1KNYeaDAuswtpJJZpPqzpuFNyvjd7LbxMXSShji
Number of transactions: 387044
Price range: 7.95e-08 to 1.15e-05

--- Testing Original Price Series ---
Augmented Dickey-Fuller Test for Token 1 Original Price


In [ ]:
# Visualize the price series for the tested tokens
print("Price Series Visualization")
print("=" * 40)

for i, token in enumerate(top_tokens):
    token_data = df[df['Token Address'] == token].copy()
    token_data = token_data.sort_values('Date')
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle(f'Token {i+1}: {token[:30]}...', fontsize=14)
    
    # Original price series
    axes[0, 0].plot(token_data['Date'], token_data['Token Price'], linewidth=0.8, alpha=0.7)
    axes[0, 0].set_title('Original Price Series')
    axes[0, 0].set_ylabel('Price')
    axes[0, 0].tick_params(axis='x', rotation=45)
    axes[0, 0].grid(True, alpha=0.3)
    
    # Log-transformed price series
    log_prices = np.log(token_data['Token Price'])
    axes[0, 1].plot(token_data['Date'], log_prices, linewidth=0.8, alpha=0.7, color='orange')
    axes[0, 1].set_title('Log-Transformed Price Series')
    axes[0, 1].set_ylabel('Log(Price)')
    axes[0, 1].tick_params(axis='x', rotation=45)
    axes[0, 1].grid(True, alpha=0.3)
    
    # First differences
    price_diff = token_data['Token Price'].diff().dropna()
    axes[1, 0].plot(token_data['Date'].iloc[1:], price_diff, linewidth=0.8, alpha=0.7, color='green')
    axes[1, 0].set_title('First Differences (Price Changes)')
    axes[1, 0].set_ylabel('Price Change')
    axes[1, 0].tick_params(axis='x', rotation=45)
    axes[1, 0].grid(True, alpha=0.3)
    
    # Histogram of first differences
    axes[1, 1].hist(price_diff, bins=50, alpha=0.7, color='red', edgecolor='black')
    axes[1, 1].set_title('Distribution of Price Changes')
    axes[1, 1].set_xlabel('Price Change')
    axes[1, 1].set_ylabel('Frequency')
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nToken {i+1} Summary:")
    print(f"  - Original prices: {'Stationary' if adf_results[f'{token}_original']['is_stationary'] else 'Non-stationary'}")
    print(f"  - Log prices: {'Stationary' if adf_results[f'{token}_log']['is_stationary'] else 'Non-stationary'}")
    print(f"  - Price differences: {'Stationary' if adf_results[f'{token}_diff']['is_stationary'] else 'Non-stationary'}")
    print()


In [ ]:
# Comprehensive ADF Test Summary
print("COMPREHENSIVE ADF TEST SUMMARY")
print("=" * 50)

# Create a summary DataFrame
summary_data = []
for key, result in adf_results.items():
    token_id, test_type = key.split('_', 1)
    summary_data.append({
        'Token': token_id[:20] + '...',
        'Test Type': test_type.title(),
        'ADF Statistic': result['adf_statistic'],
        'P-value': result['p_value'],
        'Is Stationary': result['is_stationary'],
        'Critical 1%': result['critical_values']['1%'],
        'Critical 5%': result['critical_values']['5%'],
        'Critical 10%': result['critical_values']['10%']
    })

summary_df = pd.DataFrame(summary_data)
print("\nDetailed Results:")
print(summary_df.to_string(index=False, float_format='%.6f'))

print(f"\n{'='*50}")
print("INTERPRETATION GUIDE:")
print("=" * 50)
print("• ADF Test tests the null hypothesis that a unit root is present")
print("• H0: The series has a unit root (non-stationary)")
print("• H1: The series is stationary")
print("• If p-value ≤ 0.05: Reject H0 → Series is STATIONARY")
print("• If p-value > 0.05: Fail to reject H0 → Series is NON-STATIONARY")
print("\n• For trading strategies:")
print("  - Non-stationary prices often require differencing or detrending")
print("  - Stationary series are better for mean reversion strategies")
print("  - Log transformation can help stabilize variance")
print("  - First differences (returns) are often stationary")

# Additional statistical analysis
print(f"\n{'='*50}")
print("ADDITIONAL STATISTICAL ANALYSIS")
print("=" * 50)

# Test for autocorrelation in the most active token
most_active_token = top_tokens[0]
token_data = df[df['Token Address'] == most_active_token].copy().sort_values('Date')

print(f"\nAutocorrelation Analysis for most active token:")
print(f"Token: {most_active_token[:30]}...")

# Calculate returns
returns = token_data['Token Price'].pct_change().dropna()

# Ljung-Box test for autocorrelation
try:
    lb_test = acorr_ljungbox(returns, lags=10, return_df=True)
    print(f"\nLjung-Box Test for Autocorrelation (returns):")
    print(f"Test statistic: {lb_test['lb_stat'].iloc[-1]:.4f}")
    print(f"P-value: {lb_test['lb_pvalue'].iloc[-1]:.6f}")
    
    if lb_test['lb_pvalue'].iloc[-1] < 0.05:
        print("Significant autocorrelation detected in returns")
    else:
        print("No significant autocorrelation in returns")
except Exception as e:
    print(f"Could not perform Ljung-Box test: {e}")

# Basic statistics
print(f"\nBasic Statistics for {most_active_token[:20]}...:")
print(f"Number of observations: {len(token_data)}")
print(f"Mean price: {token_data['Token Price'].mean():.2e}")
print(f"Std price: {token_data['Token Price'].std():.2e}")
print(f"Min price: {token_data['Token Price'].min():.2e}")
print(f"Max price: {token_data['Token Price'].max():.2e}")
print(f"Price volatility (CV): {token_data['Token Price'].std() / token_data['Token Price'].mean():.4f}")

print(f"\n{'='*50}")
print("ADF TEST IMPLEMENTATION COMPLETE")
print("=" * 50)
